In [1]:

all <- read.csv("../input/insurance.csv", stringsAsFactors = F)


In [2]:
library(knitr)
library(ggplot2)
library(plyr)
library(dplyr)
library(corrplot)
library(caret)
library(gridExtra)
library(scales)
library(Rmisc)
library(ggrepel)
library(randomForest)
library(psych)
library(xgboost)
library(rpart)
library('mice')
library('ggthemes')
library(glmnet)
library(foreach)

**Context**
Machine Learning with R by Brett Lantz is a book that provides an introduction to machine learning using R. As far as I can tell, Packt Publishing does not make its datasets available online unless you buy the book and create a user account which can be a problem if you are checking the book out from the library or borrowing the book from a friend. All of these datasets are in the public domain but simply needed some cleaning up and recoding to match the format in the book.

**Content**
Columns - age: age of primary beneficiary

sex: insurance contractor gender, female, male

bmi: Body mass index, providing an understanding of body, weights that are relatively high or low relative to height, objective index of body weight (kg / m ^ 2) using the ratio of height to weight, ideally 18.5 to 24.9

children: Number of children covered by health insurance / Number of dependents

smoker: Smoking

region: the beneficiary's residential area in the US, northeast, southeast, southwest, northwest.

charges: Individual medical costs billed by health insurance


**Inspiration**
Can you accurately predict insurance costs?

In [ ]:
head(all)

**1.1 Data size and structure**

In [ ]:
dim(all)

In [ ]:
str(all)

The  dataset consist of character  integer and numeric variables. Most of the character variables are factors,but R read it commits being character, we will convert all variables that are integers and characters into numeric and factor.

In [7]:
all$age<-as.numeric(all$age)
all$children<-as.numeric(all$children)
all$sex<-as.factor(all$sex)
all$smoker<-as.factor(all$smoker)
all$region<-as.factor(all$region)


In [11]:
str(all)

Great

**2 Exploring some of the most important variables

**2.1 The response variable; charge**

Let's look at how our variable is distributed, note that it is fundamental to always start looking at our target which is our target variable, it will help us to have more understanding about it. We will keep this in mind, and take measures before modeling.

In [12]:
ggplot(data=all, aes(x=all$charge)) +
        geom_histogram(binwidth=1000, fill="blue") 
summary(all$charge)

In [13]:

#let's look at the important variable
set.seed(222)
rdf<-randomForest(all$charges~.,data=all,na.action = na.roughfix)
imp_RF <- importance(rdf)
imp_DF <- data.frame(Variables = row.names(imp_RF), MSE = imp_RF[,1])
imp_DF <- imp_DF[order(imp_DF$MSE, decreasing = TRUE),]
ggplot(imp_DF[1:6,], aes(x=reorder(Variables, MSE), y=MSE, fill=MSE)) + geom_bar(stat = 'identity') + labs(x = 'Variables', y= '% increase MSE if variable is randomly permuted') + coord_flip() + theme(legend.position="none")



**3 The most important  predictors**

We will first start by exploring the factor variables and then the numerical variables.

**3.1 Factor Variable**

**3.1.1 Smoker**

In [ ]:
s1<-ggplot(all,aes(x=all$smoker,fill=all$smoker))+geom_bar(stat = 'count')+labs(x = 'people smoking') +
  geom_label(stat='count',aes(label=..count..), size=7) +theme_grey(base_size = 18)
s2<-ggplot(all, aes(x=all$smoker, y=all$charges)) +
  geom_bar(stat='summary', fun.y = "median", fill='blue') +
  theme(axis.text.x = element_text(angle = 45, hjust = 1)) +
  scale_y_continuous(breaks= seq(0, 800000, by=50000)) +
  geom_label(stat = "count", aes(label = ..count.., y = ..count..), size=3) +
  geom_hline(yintercept=9382, linetype="dashed", color = "red")
s3<-ggplot(all,aes(x=all$smoker,y=all$charges))+geom_boxplot()
grid.arrange(s1,s2,s3)


We can note that there is a significant difference between smokers and non-smokers for the insurance charges, in addition smokers are more numerous and they pay more for the charges of ensuring that non smokers.Is there a difference between sex in smokers? from what insurance charges can we know if the individual is smoking or not?
That's what we're going to visualize now

In [ ]:
s1<-ggplot(all,aes(x=all$sex,fill=all$smoker))+geom_bar(stat = 'count')+labs(x = 'people smoking') +
  geom_label(stat='count',aes(label=..count..), size=7) +theme_grey(base_size = 18)
s2<-ggplot(all,aes(x=all$charges,fill=all$smoker))+geom_density(alpha=0.5, aes(fill=factor(smoker))) + labs(title="smoker")  + theme_grey()
grid.arrange(s1,s2)

We can see in both males and females, non-smokers are much more numerous, but given the significant difference in insurance charges by smokers or non-smokers, this will leave us in doubt about the relevance of sex with respect to loads is to say is there a significant difference between males and females for insurance charges? This is what we will see when we arrive at the variable sex, for the moment we will be interested in the variable smoker.We see from the density that people who have an insurance charge of more than 1800 dollar are almost all smokers and those with less than 1800 are non-smokers

**3.1.2 Sex**

In [ ]:
s1<-ggplot(all,aes(x=all$sex,fill=all$sex))+geom_bar(stat = 'count')+labs(x = 'Sex') +
  geom_label(stat='count',aes(label=..count..), size=7) +theme_grey(base_size = 18)
s2<-ggplot(all, aes(x=all$sex, y=all$charges)) +
  geom_bar(stat='summary', fun.y = "median", fill='blue') +
  theme(axis.text.x = element_text(angle = 45, hjust = 1)) +
  scale_y_continuous(breaks= seq(0, 800000, by=50000)) +
  geom_label(stat = "count", aes(label = ..count.., y = ..count..), size=3) +
  geom_hline(yintercept=9382, linetype="dashed", color = "red")
s3<-ggplot(all,aes(x=all$sex,y=all$charges))+geom_boxplot()
grid.arrange(s1,s2,s3)

We can answer your question previously posed on the relevance of sex in relation to insurance costs, it is obvious that there is no significant difference on insurance charges according to sex.
So the insurance charges do not discriminate according to you are female or male

**3.1.3 Region**

In [ ]:
s1<-ggplot(all,aes(x=all$region,fill=all$region))+geom_bar(stat = 'count')+labs(x = 'region') +
  geom_label(stat='count',aes(label=..count..), size=7) +theme_grey(base_size = 18)
s2<-ggplot(all, aes(x=all$region, y=all$charges)) +
  geom_bar(stat='summary', fun.y = "median", fill='blue') +
  theme(axis.text.x = element_text(angle = 45, hjust = 1)) +
  scale_y_continuous(breaks= seq(0, 800000, by=50000)) +
  geom_label(stat = "count", aes(label = ..count.., y = ..count..), size=3) +
  geom_hline(yintercept=9382, linetype="dashed", color = "red")
s3<-ggplot(all,aes(x=all$region,y=all$charges))+geom_boxplot()
grid.arrange(s1,s2,s3)

We found that there is no significant difference depending on your region in relation to insurance charges, ie insurance charges do not discriminate by region where you live

**3.2 The most important numeric predictors**

**3.2.1 Correlations with charges**

Altogether, there are 4 numeric variables with a correlation of at least 0 with charge. 

In [ ]:
numericVars <- which(sapply(all, is.numeric)) #index vector numeric variables
numericVarNames <- names(numericVars) #saving names vector for use later on
cat('There are', length(numericVars), 'numeric variables')

In [ ]:
all_numVar <- all[, numericVars]
cor_numVar <- cor(all_numVar, use="pairwise.complete.obs") #correlations of all numeric variables

#sort on decreasing correlations with SalePrice
cor_sorted <- as.matrix(sort(cor_numVar[,'charges'], decreasing = TRUE))
 #select only high corelations
CorHigh <- names(which(apply(cor_sorted, 1, function(x) abs(x)>0)))
cor_numVar <- cor_numVar[CorHigh, CorHigh]

corrplot.mixed(cor_numVar, tl.col="black", tl.pos = "lt")

We find that all these numeric variables are weakly correlated with variable insurance charges.
but let's look at the variable ages a little

**3.2.2 Age**

In [ ]:
ggplot(data=all, aes(x=factor(all$age), y=all$charge))+
        geom_boxplot(col='blue') + labs(x='Overall Quality') +
        scale_y_continuous(breaks= seq(0, 800000, by=100000), labels = comma)

We can see that this positive correlation between charges and ages is real because the insurance charges increase when the person becomes more and more senior

**3.2.3 Other variables numeric**

In [ ]:
p1<-ggplot(data=all, aes(x=factor(all$age), y=all$charge))+
        geom_boxplot(col='blue') + labs(x='Age') +
        scale_y_continuous(breaks= seq(0, 800000, by=100000), labels = comma)
p2<-ggplot(data=all, aes(x=factor(all$age), y=all$charge))+
        geom_point(col='blue') + geom_smooth(method = "lm", se=FALSE, color="black", aes(group=1)) +
        scale_y_continuous(breaks= seq(0, 800000, by=100000), labels = comma)
p3<-ggplot(data=all, aes(x=factor(all$bmi), y=all$charge))+
        geom_boxplot(col='blue') + labs(x='bmi') +
        scale_y_continuous(breaks= seq(0, 800000, by=100000), labels = comma)
p4<-ggplot(data=all, aes(x=factor(all$bmi), y=all$charge))+
        geom_point(col='blue') + geom_smooth(method = "lm", se=FALSE, color="black", aes(group=1)) +
        scale_y_continuous(breaks= seq(0, 800000, by=100000), labels = comma)
p5<-ggplot(data=all, aes(x=factor(all$children), y=all$charge))+
        geom_boxplot(col='blue') + labs(x='children') +
        scale_y_continuous(breaks= seq(0, 800000, by=100000), labels = comma)
p6<-ggplot(data=all, aes(x=factor(all$children), y=all$charge))+
        geom_point(col='blue') + geom_smooth(method = "lm", se=FALSE, color="black", aes(group=1)) +
        scale_y_continuous(breaks= seq(0, 800000, by=100000), labels = comma)

grid.arrange(p1,p2,p3,p4,p5,p6)

We can see those who do not have children, we have higher insurance costs and those who have 5 children pay less

**4. Feature engineering**

**4.1 Missing Data**

let's check how much data is missing

In [ ]:
sum(is.na(all))


So there is no missing data

**4.2 Is the size of the family significant?**


In [ ]:
ggplot(all, aes(x=all$children, y=all$charges)) +
  geom_bar(stat='summary', fun.y = "median", fill='blue') +
  theme(axis.text.x = element_text(angle = 45, hjust = 1)) +
  scale_y_continuous(breaks= seq(0, 800000, by=50000)) +
  geom_label(stat = "count", aes(label = ..count.., y = ..count..), size=3) +
  geom_hline(yintercept=9382, linetype="dashed", color = "red")

We find those who have 3,4 or 0 children have a higher insurance expenses than the median so the expenses of insurance penenalise is group. We will create a variable which one will name type of family.**Let’s create a discretized family size variable**.

In [ ]:
all$childrenD[all$children==0 ]<-'type'
all$childrenD[all$children==3 ]<-'type'
all$childrenD[all$children==4]<-'type'
all$childrenD[all$children==1]<-'ptype'
all$childrenD[all$children==2 ]<-'ptype'
all$childrenD[all$children==5]<-'ptype'



In [ ]:
table(all$childrenD)

In [ ]:
ggplot(data=all, aes(x=factor(all$childrenD), y=all$charge))+
        geom_boxplot(col='blue') + labs(x='children') +
        scale_y_continuous(breaks= seq(0, 800000, by=100000), labels = comma)

**4.3 Age**

We found earlier that clouds of age points and loads their relationship is not totally linear so let's capture this non-linearity in their relationship by including a term that is square


In [ ]:
all$age2<-all$age^2

**5 Composing train and test sets**


In [ ]:
index <- sample(2,nrow(all),replace= TRUE,prob=c(0.7,0.3))
trainClean <- all[index==1,]
testClean <- all[index==2,]

**6 Modeling**

**6.1 Linear regression**


In [ ]:

model_lm <- lm(trainClean$charges~., data =trainClean)
summary(model_lm)

In [ ]:
library(forecast)
#use predict() to make prediction on a new set
pred1 <- predict(model_lm,testClean,type = "response")
residuals <- testClean$charges - pred1
linreg_pred <- data.frame("Predicted" = pred1, "Actual" = testClean$charges, "Residual" = residuals)
accuracy(pred1, testClean$charges)



** 7 What linear model is the best?**


This article will deal with two methods that slightly modify the
Ordinary least squares (OLS) regression - the regression of the ridge and the lasso.

The regression of the ridge and the lasso are closely related, but only the lasso
has the ability to choose predictors. Like OLS, Ridge tries to
minimize the residual sum of predictor squares in a given model.
However, the regression of the ridge includes an additional term of "shrinkage" - the
square of the coefficient estimate - which reduces the estimate of
coefficients to zero. The impact of this term is controlled by
another term, lambda (determined separately). Two
interesting implications of this design are the facts that when λ = 0 the
OLS coefficients are returned and when λ = ∞, the coefficients will approach
zero.

To take a look, configure a template matrix (removing the
intercept column), store the independent variable under yet create a vector of
lambda values.


In [ ]:

x <- model.matrix(all$charges~., all)[,-1]
y <- all$charges
lambda <- 10^seq(10, -2, length = 100)

In [ ]:

#Let's first prove that when λ = 0 we get the same coefficients que le modèle OLS.
#create test and training sets
set.seed(489)
train = sample(1:nrow(x), nrow(x)/2)
test = (-train)
ytest = y[test]

In [ ]:
#Adjust your models.
#OLS
chargeslm <- lm(all$charges~., data = all)
coef(chargeslm)

In [ ]:
#ridge
ridge.mod <- glmnet(x, y, alpha = 0, lambda = lambda)
coef.glmnet(ridge.mod)

In [ ]:
##The differences here are nominal. Let's see if we can use the Ridge to improve OLS estimation..
chargeslm <- lm(all$charges~., data = all, subset = train)
ridge.mod <- glmnet(x[train,], y[train], alpha = 0, lambda = lambda)
#find the best lambda from our list via cross-validation
cv.out <- cv.glmnet(x[train,], y[train], alpha = 0)

In [ ]:
bestlam <- cv.out$lambda.min
#make predictions
ridge.pred <- predict(ridge.mod, s = bestlam, newx = x[test,])
s.pred <- predict(chargeslm, newdata = all[test,])


In [ ]:
#check MSE
mean((s.pred-ytest)^2)

In [ ]:
#check MSE
mean((ridge.pred-ytest)^2)

lm works best for these data according to the MSE.

In [ ]:
#a look at the coefficients
out = glmnet(x[train,],y[train],alpha = 0)
predict(ridge.mod, type = "coefficients", s = bestlam)[1:6,]

As expected, most coefficient estimates are more conservative.

Let's take a look at the lasso. The big difference here is in the
term of narrowing - the lasso takes the absolute value of
coefficient estimates.

In [ ]:
lasso.mod <- glmnet(x[train,], y[train], alpha = 1, lambda = lambda)
lasso.pred <- predict(lasso.mod, s = bestlam, newx = x[test,])
#cheek MSE
mean((lasso.pred-ytest)^2)

The MSE is a little higher for lasso estimation. Let's look at them
coefficients.

In [ ]:
lasso.coef  <- predict(lasso.mod, type = 'coefficients', s = bestlam)[1:6,]
lasso.coef

It looks like the lasso places a big deal on smoker, bmi and age. From there we also gain a
proof that region, are not useful predictors for
this model. It is likely that sex some
effect on children, however, these coefficients because the use at zero
wrong model.
There are many other things to explore here, but I will leave the details to
experts. I am always happy to have your opinion on the topics I
speak, so feel free to leave a comment or contact me. Often,
I learn as much from you all as I do research on the subject of which I
spoken.

My previous article was on RandomForest, SVM vs. GBM model for biomechanical

https://www.kaggle.com/mahmoud86/randomforest-svm-vs-gbm-model-for-biomechanical

And I recommend you read the article fromMegan Risdal and Erik Bruin


Thanks for the reading!

**If you found this notebook helpful or you just liked it , some upvotes would be very much appreciated - That will keep me motivated to update it on a regular basis**